In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import seaborn as sns
import cvxpy as cp
import plotly.graph_objects as go
from matplotlib import pyplot as plt

### <b>Risk Free Asset </b>

In [2]:
import yfinance as yf

ticker_symbol = "^TNX"
bond_data = yf.Ticker(ticker_symbol)

historical_data = bond_data.history(period="1y")
historical_data = historical_data['Close']
simple_returns = pd.DataFrame(historical_data.pct_change().dropna())
RiskFreeRate = simple_returns.mean()[0]
RiskFreeRate

0.0005433887385254291

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

risk_free_ticker = "^TNX"  # 10-Year Treasury Yield
risk_free_data = yf.Ticker(risk_free_ticker)
risk_free_yield = risk_free_data.history(period="1y")["Close"].iloc[-1] / 100 
print("Risk-free yield:", risk_free_yield)



Risk-free yield: 0.04307000160217285


###  <b>Risky Asset Stock data</b>

In [4]:
assets = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NFLX', 'V', 'JPM', 'WMT']

data = yf.download(assets, start="2023-09-01", end="2023-12-02")['Adj Close']
newData = pd.DataFrame(yf.download(assets , start = "2023-09-01" , end = "2023-12-02")['Close'])
data = pd.DataFrame(data)
data


[*********************100%%**********************]  10 of 10 completed
[*********************100%%**********************]  10 of 10 completed


,AAPL,AMZN,GOOGL,JPM,META,MSFT,NFLX,TSLA,V,WMT
Date,,,,,,,,,,
2023-09-01,188.969589,138.119995,135.660004,144.851379,296.065887,327.388855,439.880005,245.009995,247.122101,160.972153
2023-09-05,189.208969,137.270004,135.770004,143.253098,299.831879,332.259918,448.679993,256.489990,244.363129,159.676956
2023-09-06,182.436539,135.360001,134.460007,143.016327,298.852936,331.592560,445.760010,251.919998,245.179871,160.872513
2023-09-07,177.100388,137.850006,135.259995,141.792938,298.353455,328.634033,443.140015,251.490005,246.155975,162.865112
2023-09-08,177.718781,138.229996,136.380005,141.901474,297.574280,332.977142,442.799988,248.500000,246.305359,163.164001
...,...,...,...,...,...,...,...,...,...,...
2023-11-27,189.548126,147.729996,136.410004,152.251175,334.345276,377.911133,479.170013,236.080002,253.666992,156.189911
2023-11-28,190.157349,147.029999,137.199997,152.599014,338.630707,381.993622,479.000000,246.720001,252.469223,158.052979
2023-11-29,189.128662,146.320007,134.990005,153.374252,331.847931,378.150696,477.190002,244.139999,253.756821,155.502457


In [5]:
returnData = pd.DataFrame(data.pct_change().dropna())
returnData['Mean Return'] = returnData.mean(axis = 1)
print(returnData)
mean_returns = pd.DataFrame(returnData.mean())
Market_expected_return = mean_returns.iloc[-1][0]
cov_matrix = pd.DataFrame(returnData.cov())
cov_matrix
# mean_returns

                AAPL      AMZN     GOOGL       JPM      META      MSFT   
Date                                                                     
2023-09-05  0.001267 -0.006154  0.000811 -0.011034  0.012720  0.014879  \
2023-09-06 -0.035793 -0.013914 -0.009649 -0.001653 -0.003265 -0.002009   
2023-09-07 -0.029249  0.018395  0.005950 -0.008554 -0.001671 -0.008922   
2023-09-08  0.003492  0.002757  0.008280  0.000765 -0.002612  0.013216   
2023-09-11  0.006623  0.035231  0.003959  0.004380  0.032462  0.010979   
...              ...       ...       ...       ...       ...       ...   
2023-11-27 -0.000948  0.006747 -0.002048 -0.002279 -0.010437  0.003126   
2023-11-28  0.003214 -0.004738  0.005791  0.002285  0.012817  0.010803   
2023-11-29 -0.005410 -0.004829 -0.016108  0.005080 -0.020030 -0.010060   
2023-11-30  0.003063 -0.001572 -0.018224  0.011405 -0.015202  0.000158   
2023-12-01  0.006791  0.006434 -0.005055  0.004869 -0.007122 -0.011612   

                NFLX      TSLA       

,AAPL,AMZN,GOOGL,JPM,META,MSFT,NFLX,TSLA,V,WMT,Mean Return
AAPL,0.000144,0.000130,0.000114,3.515466e-05,0.000122,9.820771e-05,0.000085,0.000117,0.000043,-7.020947e-06,0.000088
AMZN,0.000130,0.000432,0.000236,1.275355e-05,0.000264,1.355286e-04,0.000085,0.000228,0.000069,4.956298e-05,0.000164
GOOGL,0.000114,0.000236,0.000311,4.852733e-05,0.000221,9.084064e-05,0.000100,0.000172,0.000047,1.768028e-05,0.000136
JPM,0.000035,0.000013,0.000049,9.686758e-05,0.000020,2.412353e-05,0.000013,0.000060,0.000039,-4.059602e-07,0.000035
META,0.000122,0.000264,0.000221,1.951261e-05,0.000306,1.508930e-04,0.000060,0.000242,0.000040,3.015456e-05,0.000146
MSFT,0.000098,0.000136,0.000091,2.412353e-05,0.000151,1.910629e-04,0.000116,0.000127,0.000054,2.088869e-07,0.000099
NFLX,0.000085,0.000085,0.000100,1.283494e-05,0.000060,1.155837e-04,0.000664,-0.000130,0.000014,-1.005119e-05,0.000100
TSLA,0.000117,0.000228,0.000172,6.037689e-05,0.000242,1.266064e-04,-0.000130,0.000981,0.000091,1.272582e-04,0.000201
V,0.000043,0.000069,0.000047,3.888371e-05,0.000040,5.353923e-05,0.000014,0.000091,0.000094,2.095998e-05,0.000051
WMT,-0.000007,0.000050,0.000018,-4.059602e-07,0.000030,2.088869e-07,-0.000010,0.000127,0.000021,1.676870e-04,0.000040


### <b>Expected Return using CAPM formula for each asset</b>

In [6]:
covr_m = []
for x in assets:
    covr_m.append(returnData[x].cov(returnData['Mean Return']))
covr_m

[8.811967115234908e-05,
 9.865946014091676e-05,
 0.00013569617475870226,
 0.0001640771618361602,
 0.00020141488375824052,
 0.0001455123010699656,
 9.97576500476259e-05,
 5.113732096357726e-05,
 3.486288521941403e-05,
 3.9603384747411665e-05]

In [7]:
mean_returns = mean_returns.iloc[:-1].values


In [8]:
cov_matrix = cov_matrix.drop(cov_matrix.columns[-1], axis=1)
cov_matrix = cov_matrix.iloc[: -1].values
O = np.array([1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0 , 1.0])
cov_matrix

array([[ 1.43976435e-04,  1.29781476e-04,  1.13954808e-04,
         3.51546611e-05,  1.22061439e-04,  9.82077132e-05,
         8.54876017e-05,  1.16588883e-04,  4.30046411e-05,
        -7.02094681e-06],
       [ 1.29781476e-04,  4.31735770e-04,  2.35982619e-04,
         1.27535506e-05,  2.63634796e-04,  1.35528608e-04,
         8.46037313e-05,  2.27804214e-04,  6.93838721e-05,
         4.95629809e-05],
       [ 1.13954808e-04,  2.35982619e-04,  3.11068509e-04,
         4.85273316e-05,  2.20806107e-04,  9.08406416e-05,
         9.98581042e-05,  1.71621852e-04,  4.66214954e-05,
         1.76802793e-05],
       [ 3.51546611e-05,  1.27535506e-05,  4.85273316e-05,
         9.68675830e-05,  1.95126138e-05,  2.41235332e-05,
         1.28349436e-05,  6.03768860e-05,  3.88837095e-05,
        -4.05960182e-07],
       [ 1.22061439e-04,  2.63634796e-04,  2.20806107e-04,
         1.95126138e-05,  3.06123299e-04,  1.50892998e-04,
         6.02308464e-05,  2.41809064e-04,  3.98972840e-05,
         3.

In [9]:
print(cov_matrix.shape)

(10, 10)


In [10]:
numerator = np.linalg.inv(cov_matrix) @ (mean_returns - (RiskFreeRate* O.T).reshape(10 , 1))
denominator = (O @ (np.linalg.inv(cov_matrix) @ (mean_returns - (RiskFreeRate * O.T).reshape(10 , 1))))
result = numerator / denominator
print(result)

[[-21.1403484 ]
 [  6.67416467]
 [-16.55606448]
 [ 21.01432454]
 [ 13.63365324]
 [ 14.03863644]
 [  1.4134755 ]
 [ -1.0735053 ]
 [ -5.12576699]
 [-11.87856923]]


In [11]:
sigma_square = (result.T @ (cov_matrix @ result))
sigma_square[0][0]

0.12839882288831997

In [12]:
for x in range(0,len(covr_m)):
    covr_m[x] = covr_m[x]/sigma_square[0][0]
covr_m
print("Beta Values :" , covr_m)
expected_return = []
for x in range(len(covr_m)):
    val = RiskFreeRate + covr_m[x]*(Market_expected_return - RiskFreeRate)
    expected_return.append(val)
print("Expected Return  : ", expected_return)

Beta Values : [0.0006862965654209673, 0.0007683829019735624, 0.0010568334795150688, 0.0012778712307890305, 0.0015686661234692876, 0.0011332837622392438, 0.0007769358612765019, 0.00039826939074087925, 0.0002715202868311139, 0.0003084404035530629]
Expected Return  :  [0.0005435256864601031, 0.0005435420664841659, 0.0005435996257272689, 0.0005436437329908735, 0.0005437017600333691, 0.0005436148810976901, 0.0005435437731954783, 0.0005434682117054709, 0.0005434429193918025, 0.0005434502866641856]


In [13]:
dfx = pd.DataFrame({'Beta Values': covr_m, 'Expected Return': expected_return} , index = assets)
dfx = dfx.sort_values(by='Beta Values')
dfx

,Beta Values,Expected Return
JPM,0.000272,0.000543
WMT,0.000308,0.000543
V,0.000398,0.000543
AAPL,0.000686,0.000544
MSFT,0.000768,0.000544
NFLX,0.000777,0.000544
GOOGL,0.001057,0.000544
META,0.001133,0.000544
AMZN,0.001278,0.000544
TSLA,0.001569,0.000544


### <b>CAPM Expected Return</b>

In [14]:
trace1 = go.Scatter(x=covr_m, y=expected_return, mode='lines + markers' , name = "Efficient Frontier" ,text = assets,marker=dict(size = 10 , color = 'black'))

layout = go.Layout(
    title='CAPM formula for Expected Return',
    xaxis=dict(title='Beta'),
    yaxis=dict(title='Expected Return'),
    width=600  
)

fig = go.Figure(data=[trace1], layout=layout)
fig.show()


### <b>Markowitz efficient Frontier</b>

In [15]:
num_assets = 10

risk_tolerance = 0.05

weights = cp.Variable(num_assets)
risk = cp.quad_form(weights, cov_matrix)
ret = weights.T @ mean_returns

constraints = [cp.sum(weights) == 1,
               weights >= 0,
               risk <= risk_tolerance]

prob = cp.Problem(cp.Maximize(ret),
                  constraints)

prob.solve(solver = cp.ECOS)

print("Optimal weights: ", weights.value)

returns = np.linspace(min(np.array(mean_returns))[0], max(np.array(mean_returns))[0], 10000)

risks = []
count = 0
riskVal1 = 0 
returnVal1 = 0
riskVal2 = 0
returnVal2 = 0
weights1 = []
CML_Slope = 12.7467
CML_Intercept = 0.0007
for ret in returns:
    constraints = [weights.T @ mean_returns == ret,
                   cp.sum(weights) == 1,
                   weights >= 0]
    prob = cp.Problem(cp.Minimize(risk), constraints)
    prob.solve(solver = cp.ECOS)
    risks.append(prob.value)
    if(count == 7689) : 
        riskVal1 = prob.value
        returnVal1 = ret
        weights1.append(weights.value)
    if (count == 9345):
        riskVal2 = prob.value
        returnVal2 = ret
        weights1.append(weights.value)
    count = count + 1
temp = []
temp1 = []
temp2 = []
temp3 = []
for x in range(len(returns)):
    if returns[x] > 0.0006:
        temp.append(returns[x])
        temp2.append(risks[x])
    else:
        temp1.append(returns[x])
        temp3.append(risks[x])

trace1 = go.Scatter(x=temp2, y=temp, mode='markers' , name = "Efficient Frontier" ,marker=dict(size = 8 , color = 'lightgreen'))
trace2 = go.Scatter(x = temp3 , y = temp1 , mode = 'markers' , name = 'Efficient Frontier' , marker = dict(size = 8 , color = 'lightblue'))
layout = go.Layout(
    title='Markowitz Efficient frontier',
    xaxis=dict(title='Volatility'),
    yaxis=dict(title='Expected Return'),
    width=600  
)

fig = go.Figure(data=[trace1 , trace2], layout=layout)
fig.show()
# plt.scatter(temp2 , temp , color = 'lightgreen')
# plt.scatter(temp3 , temp1)
# plt.xlabel('Risk')
# plt.ylabel('Return')
# plt.title('Efficient Frontier')
# plt.show()


Optimal weights:  [1.17608858e-08 1.11028381e-08 1.16331579e-08 2.51131102e-09
 5.16844049e-09 9.99999904e-01 1.68202403e-08 1.63873057e-08
 1.02808169e-08 1.02078733e-08]


###  <b>Capital Market Line CML</b>

In [16]:
CML_slope = (max(expected_return) - risk_free_yield) / 1 
CML_intercept = risk_free_yield
CML_x = np.linspace(0, riskVal1, 100)
CML_y = CML_Slope * CML_x + CML_Intercept

### <b>Tangency Point</b>

In [17]:
print("Tangency Point Risk : " , riskVal1)
print("Tangnecy Point Return : " , returnVal1)


Tangency Point Risk :  7.247687819284784e-05
Tangnecy Point Return :  0.0015448421742694969


### <b>CML line on Efficient markowitz frontier</b>

In [18]:
import plotly.graph_objects as go
r = [riskVal1]
t = [returnVal1]
efficient_Frontier_high_Return = go.Scatter(x=temp2, y=temp, mode='markers' , name = "Efficient Frontier high return", marker=dict(size=8, color='lightgreen'))
efficient_Frontier_low_return = go.Scatter(x = temp3 , y = temp1 , mode = 'markers', name = "Efficient Frontier low return" , marker=dict(size = 8 , color = 'lightblue'))
Maximum_CAPM_return = go.Scatter(x=r , y=t , mode = "markers", name = "Maximum CAPM return", marker=dict(size = 12 , color = 'red'))
Capital_Market_Line = go.Scatter(x = CML_x , y = CML_y ,name = "Capital Market Line", mode='markers' , marker=dict(size = 4 , color = 'black'))
Risk_free_Return = go.Scatter(x=[0] , y=[0.0007] , mode = "markers" , name = "Risk Free Return" , marker=dict(size = 12 , color = 'red'))

layout = go.Layout(
    title='CML tangency with Markowitz Efficient Frontier',
    xaxis=dict(title='Risk / Volatility '),
    yaxis=dict(title='Returns'),
    width=800
)

fig = go.Figure(data=[efficient_Frontier_high_Return , efficient_Frontier_low_return , Maximum_CAPM_return , Capital_Market_Line , Risk_free_Return], layout=layout)

fig.show()

### <b>Security Market Line (SML)</b>

In [19]:
returnData1 = returnData.iloc[: , [0 , 6 ,5]]
returnData1['Mean Return'] = returnData1.mean(axis = 1)
mean_returns1 = pd.DataFrame(returnData1.mean())
Market_expected_return1 = mean_returns1.iloc[-1][0]
print(mean_returns1)
print(Market_expected_return1)
betaValues = np.linspace(-2.0 , 3.0 , 200)
print(betaValues)
expected_returnss = RiskFreeRate + betaValues*(Market_expected_return1 - RiskFreeRate)
trace = go.Scatter(x = betaValues , y = expected_returnss , mode = 'markers' , name = 'SML' , marker = dict(size = 4 , color = 'green'))
layout = go.Layout(
    title='Security Market Line',
    xaxis=dict(title='Beta '),
    yaxis=dict(title='Expected Return'), 
    width=600
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

                    0
AAPL         0.000241
NFLX         0.001216
MSFT         0.002202
Mean Return  0.001219
0.0012192379210359285
[-2.         -1.97487437 -1.94974874 -1.92462312 -1.89949749 -1.87437186
 -1.84924623 -1.8241206  -1.79899497 -1.77386935 -1.74874372 -1.72361809
 -1.69849246 -1.67336683 -1.64824121 -1.62311558 -1.59798995 -1.57286432
 -1.54773869 -1.52261307 -1.49748744 -1.47236181 -1.44723618 -1.42211055
 -1.39698492 -1.3718593  -1.34673367 -1.32160804 -1.29648241 -1.27135678
 -1.24623116 -1.22110553 -1.1959799  -1.17085427 -1.14572864 -1.12060302
 -1.09547739 -1.07035176 -1.04522613 -1.0201005  -0.99497487 -0.96984925
 -0.94472362 -0.91959799 -0.89447236 -0.86934673 -0.84422111 -0.81909548
 -0.79396985 -0.76884422 -0.74371859 -0.71859296 -0.69346734 -0.66834171
 -0.64321608 -0.61809045 -0.59296482 -0.5678392  -0.54271357 -0.51758794
 -0.49246231 -0.46733668 -0.44221106 -0.41708543 -0.3919598  -0.36683417
 -0.34170854 -0.31658291 -0.29145729 -0.26633166 -0.24120603 -0.2

C:\Users\bansa\AppData\Local\Temp\ipykernel_15880\992246466.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### <b>Sharpe Ratio && Treynor Ratio</b>

In [23]:
er = (np.array(expected_return)).reshape(10 , 1)
er_market = (er.T @ result)[0][0] - RiskFreeRate
std_dev = np.sqrt(sigma_square[0][0])
sharpe_ratio = er_market / std_dev
print("Sharpe Ratio for Optimized Portfolio : " , sharpe_ratio)
betas = (np.array(covr_m)).reshape(10 , 1)
be = (betas.T @ result)[0][0]
treynorRatio = er_market/be
print("treynor Ratio  : ",treynorRatio)

Sharpe Ratio for Optimized Portfolio :  1.8314637102669704
treynor Ratio  :  1.9954629175494962
